In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


25/03/02 18:58:09 WARN Utils: Your hostname, CairoPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/02 18:58:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/02 18:58:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/raw/green/*/*')

In [4]:
df_green.createOrReplaceTempView('green')

In [5]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 02:00:00| 244|             25.26|             2|
|2020-01-01 04:00:00| 112|             66.76|             3|
|2020-01-01 07:00:00| 169|              6.12|             1|
|2020-01-01 08:00:00| 193|              14.3|             2|
|2020-01-01 12:00:00|  25|            192.12|            11|
|2020-01-01 16:00:00| 196|36.400000000000006|             3|
|2020-01-02 11:00:00| 228|             96.46|             4|
|2020-01-02 17:00:00| 146|             15.66|             2|
|2020-01-02 20:00:00|  52|              21.3|             1|
|2020-01-03 00:00:00| 179|               7.3|             1|
|2020-01-03 05:00:00| 212|              16.8|             1|
|2020-01-03 07:00:00|  29|             163.8|             7|
|2020-01-03 08:00:00| 122|              10.8|             1|
|2020-01-03 09:00:00| 18

In [8]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/green', mode='overwrite')

25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:03:39 WARN MemoryManager: Total allocation exceeds 95.

In [9]:
df_yellow = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/raw/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/yellow', mode='overwrite')

25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:08:10 WARN MemoryManager: Total allocation exceeds 95.

In [12]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 01:00:00| 202|              42.1|             2|
|2020-01-01 02:00:00| 244|185.17000000000004|            17|
|2020-01-01 02:00:00|  71|             49.71|             1|
|2020-01-01 04:00:00|  48|           2857.21|           120|
|2020-01-01 04:00:00| 179|            119.05|             7|
|2020-01-01 06:00:00|  93|287.46000000000004|             4|
|2020-01-01 07:00:00| 169|              19.1|             2|
|2020-01-01 08:00:00| 193|            105.66|             4|
|2020-01-01 12:00:00|  25|             14.52|             1|
|2020-01-01 16:00:00| 196|             152.8|             2|
|2020-01-01 18:00:00| 157|             61.42|             1|
|2020-01-02 00:00:00| 143| 82.77999999999999|             4|
|2020-01-02 01:00:00| 262|159.72000000000003|             6|
|2020-01-02 05:00:00| 21

--------------------

In [13]:
df_green_revenue = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/yellow')

In [14]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [15]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [25]:
df_join.show(10)

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  42| 287.8700000000002|                  26| 410.8200000000002|                   38|
|2020-01-01 00:00:00|  56|23.400000000000002|                   3|              null|                 null|
|2020-01-01 00:00:00|  79|              null|                null|16100.019999999857|                  900|
|2020-01-01 00:00:00| 134|             99.06|                   6|              null|                 null|
|2020-01-01 00:00:00| 145|              69.1|                   4| 409.0000000000001|                   22|
|2020-01-01 00:00:00| 163|              null|                null|2977.1900000000055|                  154|
|2020-01-01 00:00:00| 228|  

In [16]:
df_join.write.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/total', mode='overwrite')

25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers


25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 19:15:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


-----------------

In [17]:
df_total = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/report/revenue/total')

In [18]:
df_total

DataFrame[hour: timestamp, zone: bigint, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [24]:
df_total.show(5)

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  43|             20.85|                   2|365.40000000000003|                   22|
|2020-01-01 00:00:00|  47|41.239999999999995|                   2|29.900000000000002|                    3|
|2020-01-01 00:00:00|  52|             18.05|                   1|            121.09|                    5|
|2020-01-01 00:00:00|  70|              null|                null|             42.72|                    2|
|2020-01-01 00:00:00|  85|              null|                null|              62.7|                    1|
+-------------------+----+------------------+--------------------+------------------+---------------------+
only showing top 5 rows



In [28]:
df_zones = spark.read.parquet('/home/de-zoomcamp-2025/5_batch/data/zones/')

In [29]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [30]:
df_result = df_total.join(df_zones, df_total.zone == df_zones.LocationID)

In [31]:
df_result.drop('LocationID', 'zone').write.parquet('/home/de-zoomcamp-2025/5_batch/data/tmp/revenue-zones')

25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


25/03/02 20:00:51 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 20:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 20:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/03/02 20:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/03/02 20:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/03/02 20:00:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [32]:
df_result.show(5)

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|LocationID|  Borough|              Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+------------------+------------+
|2020-01-01 00:00:00|  43|             20.85|                   2|365.40000000000003|                   22|        43|Manhattan|      Central Park| Yellow Zone|
|2020-01-01 00:00:00|  47|41.239999999999995|                   2|29.900000000000002|                    3|        47|    Bronx|Claremont/Bathgate|   Boro Zone|
|2020-01-01 00:00:00|  52|             18.05|                   1|            121.09|                    5|        52| Brooklyn|       Cobble Hill|   Boro Zone|
|2020-01-01 00:00:00|  70|        